# Stock market prediction with LSTM


In this notebook I tried to predict US stock market price with S&P 500 with tensorflow and LSTM Networks

In [89]:
import numpy as np
import os
import random
import re
import shutil
import time
import tensorflow as tf
import inspect
import matplotlib.pyplot as plt

from tensorflow.contrib.tensorboard.plugins import projector

In [8]:
import pandas as pd
import random

In [171]:
raw_df = pd.read_csv("stock_from_1900.csv")


In [172]:
raw_seq = raw_df['Close'].tolist()
raw_seq = np.array(raw_seq)

In [173]:
raw_seq = [np.array(raw_seq[i * input_size: (i + 1) * input_size])for i in range(len(raw_seq) //input_size)]
X = np.array([raw_seq[i: i + 5] for i in range(len(raw_seq) - 5)])
y = np.array([raw_seq[i + 5] for i in range(len(raw_seq) -5)])
train_size = int(len(X) * (1.0 - 0.1))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

In [177]:
lstm_size=128
keep_prob=0.8
init_learning_rate = 0.001
learning_rate_decay=0.99
num_steps=5
input_size=1
num_epochs = 1500
num_layers=6
init_epoch=5

In [178]:
tf.reset_default_graph()

In [179]:
learning_rate = tf.placeholder(tf.float32, None, name="learning_rate")
inputs = tf.placeholder(tf.float32, [None, num_steps, input_size], name="inputs")
targets = tf.placeholder(tf.float32, [None, input_size], name="targets")

def createcell():
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True)
    if keep_prob < 1.0:
        lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    return lstm_cell

cell = tf.contrib.rnn.MultiRNNCell([createcell() for _ in range(num_layers)],state_is_tuple=True) 
val, _ = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, scope="dynamic_rnn")
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1, name="last")
ws = tf.Variable(tf.truncated_normal([lstm_size, input_size]), name="w")
bias = tf.Variable(tf.constant(0.1, shape=[input_size]), name="b")
pred = tf.matmul(last, ws) + bias
loss = tf.reduce_mean(tf.square(pred - targets), name="loss")
optim = tf.train.RMSPropOptimizer(learning_rate).minimize(loss, name="optim")
test_data_feed = {
    inputs: test_X,
    targets: test_y}
init = tf.global_variables_initializer()
sess = tf.Session()
epoch_cost = 0.
sess.run(init)
for epoch in range(num_epochs):
    learning_rate = 0.0001   
    _ , epoch_cost = sess.run([pred, loss], test_data_feed)
    if  epoch % 100 == 0:
        print ("Cost after epoch %i: %f" % (epoch, epoch_cost))

sess.close()

C:\Users\titus\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Cost after epoch 0: 4928956.000000
Cost after epoch 100: 4928958.500000
Cost after epoch 200: 4928959.000000
Cost after epoch 300: 4928958.500000
Cost after epoch 400: 4928959.000000
Cost after epoch 500: 4928956.000000
Cost after epoch 600: 4928960.000000


KeyboardInterrupt: 

Seems the loss did not decrease with loop, I will make more research on that